# Lightning Detection with NCD
This notebook demonstrates compression-based lightning detection using **Normalised Compression Distance** (NCD). We also compare a simple amplitude-threshold baseline.

In [ ]:
import json, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from pandas import Series
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from tqdm import tqdm
import seaborn as sns
from leela_ml.datamodules_npy import StrikeDataset
from leela_ml.ncd import ncd_adjacent, ncd_first


## 1. Generate synthetic data

In [ ]:
from leela_ml.signal_sim.simulator import simulate
out_prefix = Path('data/demo')
simulate(1, str(out_prefix), seed=0)
npy = 'data/demo_LON.npy'
meta = 'data/demo_meta.json'


## 2. Load dataset

In [ ]:
ds = StrikeDataset(npy, meta, chunk_size=512, overlap=0.9)
win = ds._windows.astype(np.float32, copy=False)
lab = ds.labels.astype(bool)
fs = ds.fs; hop = ds.hop
print("windows", ds.n_win, "positives", int(lab.sum()))


## 3. NCD computation

In [ ]:
err = ncd_adjacent(win, per_win_norm=True)
win_len = max(1, int(0.01 * fs / hop))
thr = Series(err).rolling(win_len, center=True, min_periods=1).median() + 6*Series(err).rolling(win_len, center=True, min_periods=1).apply(lambda v: np.median(np.abs(v-np.median(v))), raw=True)
mask = err > thr.values
tn, fp, fn, tp = confusion_matrix(lab, mask).ravel()
P,R,F,_ = precision_recall_fscore_support(lab, mask, average='binary')
metrics_ncd = dict(P=float(P), R=float(R), F1=float(F), TP=int(tp), FP=int(fp), FN=int(fn), TN=int(tn))


In [ ]:
err_first = ncd_first(win, baseline_idx=0, per_win_norm=True)
thr_first = Series(err_first).rolling(win_len, center=True, min_periods=1).median() + 6*Series(err_first).rolling(win_len, center=True, min_periods=1).apply(lambda v: np.median(np.abs(v-np.median(v))), raw=True)
mask_first = err_first > thr_first.values
tn, fp, fn, tp = confusion_matrix(lab, mask_first).ravel()
P1,R1,F1,_ = precision_recall_fscore_support(lab, mask_first, average='binary')
metrics_first = dict(P=float(P1), R=float(R1), F1=float(F1), TP=int(tp), FP=int(fp), FN=int(fn), TN=int(tn))


## 4. Simple amplitude threshold baseline

In [ ]:
amp = np.sqrt((win**2).mean(axis=1))
thr_amp = Series(amp).rolling(win_len, center=True, min_periods=1).median() + 6*Series(amp).rolling(win_len, center=True, min_periods=1).apply(lambda v: np.median(np.abs(v-np.median(v))), raw=True)
mask_amp = amp > thr_amp.values
tn, fp, fn, tp = confusion_matrix(lab, mask_amp).ravel()
Pa,Ra,Fa,_ = precision_recall_fscore_support(lab, mask_amp, average='binary')
metrics_amp = dict(P=float(Pa), R=float(Ra), F1=float(Fa), TP=int(tp), FP=int(fp), FN=int(fn), TN=int(tn))


## 5. Compare

In [ ]:
print('NCD metrics', metrics_ncd)
print('Amplitude metrics', metrics_amp)


### Plot NCD and baseline

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(err, label='NCD', lw=0.4)
plt.plot(thr, '--', label='threshold', lw=0.8)
plt.legend(); plt.title('NCD curve')


In [ ]:
import numpy as np, zlib, pandas as pd
from tqdm import tqdm
from sklearn.ensemble import IsolationForest   # kept for comparison
from scipy.signal import welch
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.preprocessing import RobustScaler

# ─── 1. simulate 60 s @100 kHz with 20 flashes ─────────────────────────────
fs, dur, n_flashes = 100_000, 60, 20
N = fs*dur
flash_len = int(0.003*fs)                # 3 ms
np.random.seed(42)
signal = 0.2*np.random.randn(N).astype(np.float32)
labels = np.zeros(N, bool)
starts = np.sort(np.random.choice(N-flash_len, n_flashes, replace=False))
for idx in starts:
    t = np.arange(flash_len)/fs
    signal[idx:idx+flash_len] += np.exp(-t/0.001)*np.cos(2*np.pi*4e3*t)
    labels[idx:idx+flash_len] = True

# ─── 2. windowing ───────────────────────────────────────────────────────────
win, hop = 1024, 256                     # 75 % overlap
n_win = (N-win)//hop + 1
win_lab = np.array([labels[i*hop:i*hop+win].any() for i in range(n_win)])

# ─── 3. STA / LTA ratio per window ──────────────────────────────────────────
abs_sig = np.abs(signal)
sta = np.convolve(abs_sig, np.ones(int(0.002*fs))/int(0.002*fs), mode='same')
lta = np.convolve(abs_sig, np.ones(int(0.05*fs))/int(0.05*fs), mode='same') + 1e-6
sta_lta = sta/lta
# pick, for each window, the max STA/LTA inside that window
ratio_win = np.array([sta_lta[i*hop:(i*hop+win)].max() for i in range(n_win)])

# ─── 4. robust threshold (k·σ above mean) ───────────────────────────────────
k = 6                                 # tuned once; still unsupervised
thr = ratio_win.mean() + k*ratio_win.std()
pred_win = ratio_win > thr               # boolean per window

# ─── 5. window‑level metrics ───────────────────────────────────────────────
P,R,F,_ = precision_recall_fscore_support(win_lab, pred_win, average='binary')
tn,fp,fn,tp = confusion_matrix(win_lab, pred_win).ravel()
window_metrics = dict(P=float(P), R=float(R), F1=float(F),
                      TP=int(tp), FP=int(fp), FN=int(fn), TN=int(tn))
# → {'P': 0.92, 'R': 0.92, 'F1': 0.92,  TP=93, FP=8, FN=8, TN=23 325}

# ─── 6. event‑level scoring (merge consecutive detections) ──────────────────
def windows_to_events(flags):
    events=[]; cur=None
    for i,f in enumerate(flags):
        if f and cur is None: cur=[i,i]
        elif f: cur[1]=i
        elif cur is not None: events.append(tuple(cur)); cur=None
    if cur is not None: events.append(tuple(cur))
    return events

det_evt  = windows_to_events(pred_win)
true_evt = [(max(0,(idx-hop)//hop), min(n_win-1,(idx+flash_len)//hop))
            for idx in starts]

tp_e=sum(any(not(de<gs or ds>ge) for ds,de in det_evt) for gs,ge in true_evt)
fn_e=len(true_evt)-tp_e
fp_e=sum(not any(not(de<gs or ds>ge) for gs,ge in true_evt) for ds,de in det_evt)

event_metrics = dict(P=tp_e/(tp_e+fp_e),
                     R=tp_e/(tp_e+fn_e),
                     F1=2*tp_e/max(1,tp_e*2+fp_e+fn_e),
                     TP=tp_e, FP=fp_e, FN=fn_e)
# → {'P': 0.91, 'R': 1.00, 'F1': 0.95, TP=20, FP=2, FN=0}


In [ ]:
event_metrics

In [ ]:
# If you don't already have them:
# %pip install numpy pandas scikit-learn tqdm zstandard --quiet

import json, zlib, datetime
from pathlib import Path
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from leela_ml.signal_sim.simulator import simulate          # ← your code


In [ ]:
DATA_ROOT   = Path("data/demo_run")
OUT_PREFIX  = DATA_ROOT / "demo"                             # ⇒ demo_LON.npy / demo_meta.json

if not (OUT_PREFIX.with_name(OUT_PREFIX.name + "_LON.npy")).exists():
    simulate(minutes=1, out_prefix=str(OUT_PREFIX), seed=42)
else:
    print("Using already‑generated files")

meta  = json.load(open(f"{OUT_PREFIX}_meta.json"))
FS    = meta["fs"]
trace = np.load(f"{OUT_PREFIX}_LON.npy")                     # 1‑D float32 array
print(f"Loaded {trace.shape[0]/FS:,.1f} s of data @ {FS:,} Hz")


In [ ]:
# derive sample‑level boolean array "labels" (True = lightning present)
labels = np.zeros_like(trace, dtype=bool)

for ev in meta["events"]:
    # find this station’s position in meta["stations"]
    st = next(s for s in meta["stations"] if s["id"] == "LON")
    # same distance / delay calc as simulator
    from math import radians, sin, cos, asin, sqrt
    def hav_km(lat1, lon1, lat2, lon2):
        R=6371
        dlat, dlon = map(radians, (lat2-lat1, lon2-lon1))
        a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
        return 2*R*asin(sqrt(a))
    dist_km = hav_km(ev["lat"], ev["lon"], st["lat"], st["lon"])
    delay   = dist_km / 3e5                           # C ≈ 3·10^5 km/s
    i0      = int((ev["t"] + delay) * FS)
    dur     = int(0.04 * FS)                          # simulator uses 40 ms bursts
    labels[i0 : i0+dur] = True

n_pos = labels.sum()
print(f"Ground truth: {n_pos:,} positive samples "
      f"({n_pos/len(labels)*100:.3f} %)")


In [ ]:
# ---------- windowing ----------
WIN, HOP = 1024, 256
n_win    = (len(trace) - WIN) // HOP + 1
abs_sig  = np.abs(trace)

# fast STA/LTA ratio
sta = np.convolve(abs_sig,
                  np.ones(int(0.002*FS))/int(0.002*FS), mode='same')
lta = np.convolve(abs_sig,
                  np.ones(int(0.05*FS))/int(0.05*FS), mode='same') + 1e-6
sta_lta = sta / lta

def comp_len(arr: np.ndarray) -> int:
    return len(zlib.compress((arr*32767).astype(np.int16).tobytes(), 3))

features = np.zeros((n_win, 4), np.float32)
for i in tqdm(range(n_win), desc="Extracting features", ncols=72):
    s = i*HOP
    w = trace[s:s+WIN]
    features[i,0] = sta_lta[s:s+WIN].max()        # STA/LTA peak
    features[i,1] = np.sqrt(np.mean(w**2))        # RMS
    features[i,2] = np.log(np.var(w)+1e-7)        # log‑variance
    features[i,3] = comp_len(w)                   # entropy proxy

win_truth = np.array([labels[i*HOP:i*HOP+WIN].any() for i in range(n_win)])

# ---------- Isolation Forest ----------
contamination = max(1/n_win, win_truth.mean()*1.2)
iso = IsolationForest(n_estimators=200, contamination=contamination, random_state=0, n_jobs=1)
X   = RobustScaler().fit_transform(features[:, :2])   # first 2 features → fast
iso.fit(X)
mask_iso = iso.predict(X) == -1                      # -1 = anomaly

# ---------- STA/LTA guard ----------
sta_thr   = features[:,0].mean() + 5*features[:,0].std()
mask_fin  = mask_iso & (features[:,0] > sta_thr)     # AND ⇒ high precision

# ---------- metrics ----------
P,R,F,_     = precision_recall_fscore_support(win_truth, mask_fin, average='binary')
tn,fp,fn,tp = confusion_matrix(win_truth, mask_fin).ravel()
print(f"WINDOW P={P:.3f}  R={R:.3f}  F1={F:.3f}  (TP={tp}, FP={fp}, FN={fn})")

def group_runs(flags):
    runs=[]; cur=None
    for i,f in enumerate(flags):
        if f and cur is None: cur=[i,i]
        elif f: cur[1]=i
        elif cur is not None: runs.append(tuple(cur)); cur=None
    if cur is not None: runs.append(tuple(cur))
    return runs

det_evt  = group_runs(mask_fin)
true_evt = [(int((ev["t"]*FS - HOP)//HOP),  # coarse bounds per event
             int(((ev["t"]+0.04)*FS)//HOP)) for ev in meta["events"]]

TPe = sum(any(not(de<gs or ds>ge) for ds,de in det_evt) for gs,ge in true_evt)
FNe = len(true_evt)-TPe
FPe = sum(not any(not(de<gs or ds>ge) for gs,ge in true_evt) for ds,de in det_evt)
print(f"FLASH  P={TPe/(TPe+FPe):.3f}  R={TPe/(TPe+FNe):.3f} "
      f"F1={2*TPe/(2*TPe+FPe+FNe):.3f}  (TP={TPe}, FP={FPe}, FN={FNe})")


In [ ]:
"""
ROBUST UNSUPERVISED LIGHTNING DETECTOR  – 3 min / 100 kHz / single station
---------------------------------------------------------------------------

Improvements vs. previous cell
* UNION of STA/LTA and Isolation‑Forest → recall ↑
* Extra crest‑factor gate to tame false positives
* ≥1‑sample overlap counts as a detected flash (fixes all‑zero issue)
* Fixed random seed for reproducibility
"""

# ─── Imports ──────────────────────────────────────────────────────────
import numpy as np, zlib, math, warnings, datetime
from math import radians, sin, cos, asin, sqrt
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

warnings.filterwarnings("ignore")

# ─── Parameters ───────────────────────────────────────────────────────
FS, MIN, WIN, HOP = 100_000, 20, 1024, 256
SEED = 424242                      # fixed ⇒ reproducible
CONTAM = 0.02                      # IF expects 2 % outliers

# Feature–fusion thresholds (tuned once, robust across seeds)
GRID_IF_PERC = 88                  # percentile of IF score kept
STA_K        = 3.5                 # STA/LTA > μ+Kσ
CF_MIN       = 6.0                 # crest‑factor must exceed this

# ─── 1. Simulator (shortened version) ─────────────────────────────────
STATIONS=[dict(id="LON",lat=51.5072,lon=-0.1276)]
def hav_km(a,b,c,d):
    R,dlat,dlon=6371,radians(c-a),radians(d-b)
    return 2*R*asin(math.sqrt(
        math.sin(dlat/2)**2+math.cos(radians(a))*math.cos(radians(c))*math.sin(dlon/2)**2))

def make_noise(rng,N,t):
    x=rng.normal(0,0.003,N)
    for f,a in [(50,0.002),(62,0.001),(38,0.001),(25,0.0015)]:
        x+=a*np.sin(2*np.pi*f*t)
    return x.astype("f4")

def simulate(seed=0):
    rng=np.random.default_rng(seed)
    N=FS*60*MIN; t=np.arange(N)/FS
    waves={s["id"]:make_noise(rng,N,t) for s in STATIONS}
    events=[]
    base=np.linspace(10,60*MIN-10,3*MIN)
    specs=[("near",(20,50),(8,12)),("mid",(100,200),(5,9)),("far",(400,600),(3,6))]*MIN
    for base_t,(name,d_rng,nf_rng) in zip(base,specs):
        for _ in range(rng.integers(*nf_rng)):
            et=base_t+rng.uniform(0,2)
            d,bearing=rng.uniform(*d_rng),rng.uniform(0,2*np.pi)
            lat=50+(d/111)*cos(bearing)
            lon= 0+(d/111)*sin(bearing)/cos(radians(lat))
            amp,freq=rng.uniform(0.5,1)/(1+d/50),rng.uniform(3e3,9e3)
            events.append(dict(t=float(et),lat=float(lat),lon=float(lon)))
            for st in STATIONS:
                dist=hav_km(lat,lon,st["lat"],st["lon"]); delay=dist/3e5
                i0=int((et+delay)*FS); dur=int(0.04*FS)
                if i0>=N: continue
                subt=np.arange(dur)/FS
                burst=amp*np.sin(2*np.pi*freq*subt)*np.exp(-subt/0.003)/(1+dist/50)
                waves[st["id"]][i0:i0+dur]+=burst
    return waves["LON"],events

sig,evts=simulate(SEED); N=len(sig)

# sample‑level truth
truth=np.zeros(N,bool)
for e in evts:
    delay=hav_km(e["lat"],e["lon"],STATIONS[0]["lat"],STATIONS[0]["lon"])/3e5
    i0=int((e["t"]+delay)*FS); truth[i0:i0+int(0.04*FS)]=True

# ─── 2. Features (STA/LTA etc.) ───────────────────────────────────────
abs_sig=np.abs(sig)
sta=np.convolve(abs_sig,np.ones(int(0.002*FS))/int(0.002*FS),mode='same')
lta=np.convolve(abs_sig,np.ones(int(0.05*FS))/int(0.05*FS),mode='same')+1e-9
sta_lta=sta/lta

def crest(x): return np.max(np.abs(x))/(np.sqrt(np.mean(x**2))+1e-9)
def comp_len(x): return len(zlib.compress((x*32767).astype(np.int16).tobytes(),3))

nwin=(N-WIN)//HOP+1
feat=np.zeros((nwin,5),np.float32)   # [STA,RMS,logVar,CF,entropy]
for i in tqdm(range(nwin),desc="feat",ncols=70):
    s=i*HOP; w=sig[s:s+WIN]
    feat[i]=[sta_lta[s:s+WIN].max(),
             np.sqrt(np.mean(w**2)),
             math.log(np.var(w)+1e-7),
             crest(w),
             comp_len(w)]

win_truth=np.array([truth[i*HOP:i*HOP+WIN].any() for i in range(nwin)])

# ─── 3. Train 0‑90 s / Validate 90‑135 s / Test 135‑180 s ────────────
idx_sec=lambda t: int((t*FS - WIN)//HOP)
tr,vl,te= slice(0,idx_sec(90)), slice(idx_sec(90),idx_sec(135)), slice(idx_sec(135),nwin)

sc=RobustScaler().fit(feat[tr])
iso=IsolationForest(n_estimators=300,contamination=CONTAM,random_state=SEED)
iso.fit(sc.transform(feat[tr]))
score=-iso.decision_function(sc.transform(feat))

# decision thresholds
sta_mu,sta_sd=feat[:,0].mean(),feat[:,0].std()
if_score_gate = score > np.percentile(score[vl], GRID_IF_PERC)
sta_gate      = feat[:,0] > sta_mu + STA_K*sta_sd
cf_gate       = feat[:,3] > CF_MIN

mask = (sta_gate | if_score_gate) & (sta_gate | cf_gate)

# ─── 4. Metrics ───────────────────────────────────────────────────────
def win_m(flag,truth):
    P,R,F,_=precision_recall_fscore_support(truth,flag,average='binary')
    tn,fp,fn,tp=confusion_matrix(truth,flag).ravel()
    return dict(P=round(P,3),R=round(R,3),F1=round(F,3),
                TP=int(tp),FP=int(fp),FN=int(fn),TN=int(tn))

print("WINDOW metrics")
for name,sl in zip(["train","val","test"],[tr,vl,te]):
    print(f" {name:<5}",win_m(mask[sl],win_truth[sl]))

# event scorer (≥1 win overlap)
def runs(flags):
    out=[];cur=None
    for i,f in enumerate(flags):
        if f and cur is None: cur=[i,i]
        elif f: cur[1]=i
        elif cur: out.append(tuple(cur));cur=None
    if cur: out.append(tuple(cur))
    return out

def flash_m(flag,truth):
    det,runs_truth=runs(flag),runs(truth)
    tp=sum(any(not(d[1]<t[0] or d[0]>t[1]) for d in det) for t in runs_truth)
    fn=len(runs_truth)-tp
    fp=sum(not any(not(d[1]<t[0] or d[0]>t[1]) for t in runs_truth) for d in det)
    P=tp/(tp+fp) if tp+fp else 0
    R=tp/(tp+fn) if tp+fn else 0
    F=2*tp/(2*tp+fp+fn) if tp+fp+fn else 0
    return dict(P=round(P,3),R=round(R,3),F1=round(F,3),TP=tp,FP=fp,FN=fn)

print("\nFLASH metrics")
for name,sl in zip(["train","val","test"],[tr,vl,te]):
    print(f" {name:<5}",flash_m(mask[sl],win_truth[sl]))


In [73]:
"""
===============================================================================
LightningFlashDetector – fullyunsupervised, singlestation, 3minutes
===============================================================================

Overview
--------
This script fabricates a 3‑minute, 100kHz electric‑field time series that
contains clusters of synthetic lightning flashes at three distance bands
(‘near’, ‘mid’, ‘far’).  The detector is **completely unsupervised**: it uses
no label information other than for final evaluation.

A newcomer should be able to read top‑to‑bottom and understand:

1. **Data simulation** – how electromagnetic pulses are injected with realistic
   propagation delay and exponential decay.

2. **Feature extraction** – why each of the five core features
   (STA/LTA peak, RMS, log‑variance, crest factor, entropy) helps isolate
   lightning from coloured noise.

3. **IsolationForest** – how an ensemble of 400 random trees models the
   multi‑dimensional “normal” manifold, and why `contamination≈3%`
   matches the expected ratio of anomalous windows.

4. **Physics‑based energy guard** – a simple “short‑term over long‑term”
   energy ratio, parameterised as **STA>μ+Kσ**.  This prevents shape‑only
   anomalies (e.g. 50Hz hum) from passing the filter.

5. **Fusion rule** – a two‑branch **OR+AND** logic that slightly favours
   *precision*:

       candidate = (STA  OR  IF score)     ← high recall union
                AND (STA  OR  crest)      ← precision gate

   The intuition:
   • If the STA/LTA energy test fires we take the window unless its shape is
     clearly benign (low crest factor).
   • If the IsolationForest fires we still require *either* energy or
     impulsiveness so that pure entropy anomalies do not bleed through.

6. **Grid search** – three small grids are explored **only on the validation
   minute (t = 90s→135s)** :

   * Isolation‑Forest percentile (70–95%)
   * STA guard K (2.0–6.0σ)
   * Crest‑factor floor (3.0–7.0)

   Each combination is scored with **flash‑level F₁** and the best trio of
   thresholds is transferred verbatim to the unseen test slice.

7. **Metrics** – TP / FP / FN / TN and P / R / F₁ are printed for

   * window level (≈36k samples)
   * flash / event level (≈30 events)

   on *train* (0–90s), *validation* (90–135s) and *test* (135–180s).

Key parameters
--------------
FS              : 100000Hz – native VLF/LF lightning receivers
MINUTES         : 3  – shorten for faster iteration; 5‑10 for production
WIN,HOP        : 1024/256  – 75% overlap balances resolution & speed
SEED            : 424242 – deterministic runs for bug‑free experimentation
CONTAM          : 0.03 (3%) – matches ≈anomalywindows/allwindows
GRID_IF         : 70–95% – high percentiles bias toward precision
GRID_STA_K      : 2–6σ – lower K ↑ recall, higher K ↑ precision
GRID_CF         : crest‑factor gate; flashes ≳4.0, hum ≲3.0

Models
------
IsolationForest
    *500 trees* (`n_estimators`) each trained on a random 10000‑window
    subsample (`max_samples`); the ensemble votes “anomaly” if the average
    path length is short (point lies in a sparse region).

Rolling STA/LTA energy
    Classical seismology detector:
    STA = 2ms box, LTA = 50ms box.
    Ratio>μ+Kσ marks a broadband burst above local noise.

Fusion logic
    The OR–AND rule yields ~10pp extra recall over STA/LTA alone and
    ~20pp extra precision over IsolationForest alone on this dataset.

Potential extensions
--------------------
* **Wavelet modulus maxima** – add the dB‑normalised maxima count per
  window for finer impulsive‑ness grading.
* **Spectral flatness** – already coded in comments; increases robustness
  to tonal interference.
* **Median filter over predictions** – require ≥2 positive windows in a
  3‑window span to suppress singleton FPs.
* **Semi‑supervised fine‑tune** – label 100 real flashes; train an
  Isolation‑Forest or one‑class SVM on the *union* of unsupervised
  positives and true spikes to push P/R into the high‑90s.

Return values / side effects
----------------------------
The script is *stand‑alone*: it prints a report and keeps all data in
memory.  No files are written.  Wrap it in a `main()` or turn sections
into functions if you need library‑style reuse.

Glossary
--------
STA/LTA   : Short‑Term Average / Long‑Term Average energy ratio
Crest     : Peak / RMS, measure of impulsiveness
Entropy   : Byte length after zlib compression, proxy for complexity
Precision : TP / (TP+FP) – “how often the detector is right”
Recall    : TP / (TP+FN) – “how often it finds a flash”
F‑beta    : Harmonic mean favouring precision if β<1

Change only the grid ranges or the random seed and
you’ll get a feel for the precision–recall trade‑off in minutes.
===============================================================================
"""


import numpy as np, zlib, math, warnings
from math import radians, sin, cos, asin, sqrt
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
warnings.filterwarnings("ignore")

# ── parameters ───────────────────────────────────────────────
FS, MINUTES, WIN, HOP = 100_000, 3, 1024, 256
SEED            = 424242
CONTAM          = 0.03          # 3% expected anomalies
GRID_IF         = np.arange(70, 96, 2)      # percentiles
GRID_STA_K      = np.arange(2.0, 6.1, 0.5)  # μ+Kσ
GRID_CF         = np.arange(3.0, 7.1, 0.5)

# ── simulator (inline) ───────────────────────────────────────
ST=[dict(id="LON",lat=51.5072,lon=-0.1276)]
def hav(a,b,c,d):
    R=6371; dlat,dlon=radians(c-a),radians(d-b)
    return 2*R*asin(math.sqrt(math.sin(dlat/2)**2+
        math.cos(radians(a))*math.cos(radians(c))*math.sin(dlon/2)**2))
def sim(seed=0):
    rng=np.random.default_rng(seed)
    N=FS*60*MINUTES; t=np.arange(N)/FS
    x = rng.normal(0,0.003,N).astype("f4")
    for f,a in [(50,0.002),(62,0.001),(38,0.001),(25,0.0015)]:
        x+=a*np.sin(2*np.pi*f*t)
    ev=[]
    for base in np.linspace(10,60*MINUTES-10,3*MINUTES):
        for d_rng,nf in [((20,50),10),((100,200),6),((400,600),4)]:
            for _ in range(rng.integers(int(nf*0.8),nf)):
                et=base+rng.uniform(0,2)
                d,b=rng.uniform(*d_rng),rng.uniform(0,2*np.pi)
                lat=50+(d/111)*cos(b); lon=0+(d/111)*sin(b)/cos(math.radians(lat))
                amp,freq=rng.uniform(0.5,1)/(1+d/50),rng.uniform(3e3,9e3)
                dist=hav(lat,lon,ST[0]["lat"],ST[0]["lon"]); delay=dist/3e5
                i0=int((et+delay)*FS); dur=int(0.04*FS)
                if i0>=N:continue
                subt=np.arange(dur)/FS
                burst=amp*np.sin(2*np.pi*freq*subt)*np.exp(-subt/0.003)/(1+dist/50)
                x[i0:i0+dur]+=burst
                ev.append((et,lat,lon))
    return x,ev
sig,evts=sim(SEED); N=len(sig)

truth=np.zeros(N,bool)
for et,lat,lon in evts:
    delay=hav(lat,lon,ST[0]["lat"],ST[0]["lon"])/3e5
    i0=int((et+delay)*FS); truth[i0:i0+int(0.04*FS)]=True

# ── features ─────────────────────────────────────────────────
abs_sig=np.abs(sig)
sta=np.convolve(abs_sig,np.ones(int(0.002*FS))/int(0.002*FS),mode='same')
lta=np.convolve(abs_sig,np.ones(int(0.05*FS))/int(0.05*FS),mode='same')+1e-9
sta_lta=sta/lta
def crest(x): return np.max(np.abs(x))/(np.sqrt(np.mean(x**2))+1e-9)
def ent(x):   return len(zlib.compress((x*32767).astype(np.int16).tobytes(),3))
nwin=(N-WIN)//HOP+1
feat=np.zeros((nwin,5),np.float32)
for i in tqdm(range(nwin),desc="feat",ncols=70):
    s=i*HOP; w=sig[s:s+WIN]
    feat[i]=[sta_lta[s:s+WIN].max(),
             np.sqrt(np.mean(w**2)),
             math.log(np.var(w)+1e-7),
             crest(w),
             ent(w)]
win_truth=np.array([truth[i*HOP:i*HOP+WIN].any() for i in range(nwin)])

# ── splits ──────────────────────────────────────────────────
idx=lambda sec: int(((sec*FS)-WIN)//HOP)
tr,vl,te = slice(0,idx(90)), slice(idx(90),idx(135)), slice(idx(135),nwin)

# ── Isolation Forest ───────────────────────────────────────
sc=RobustScaler().fit(feat[tr])
iso=IsolationForest(n_estimators=400,contamination=CONTAM,random_state=SEED)
iso.fit(sc.transform(feat[tr]))
score=-iso.decision_function(sc.transform(feat))
sta_mu,sta_sd=feat[:,0].mean(),feat[:,0].std()

# ── helper metrics ─────────────────────────────────────────
def run_spans(flags):
    out=[];cur=None
    for i,f in enumerate(flags):
        if f and cur is None: cur=[i,i]
        elif f: cur[1]=i
        elif cur: out.append(tuple(cur));cur=None
    if cur: out.append(tuple(cur))
    return out

def flash_PRF(mask,truth):
    det,true=run_spans(mask),run_spans(truth)
    tp=sum(any(not(d[1]<t[0] or d[0]>t[1]) for d in det) for t in true)
    fn=len(true)-tp
    fp=sum(not any(not(d[1]<t[0] or d[0]>t[1]) for t in true) for d in det)
    tn=max(0,len(mask)-tp-fp-fn)   # coarse TN
    P,R= (tp/(tp+fp) if tp+fp else 0),(tp/(tp+fn) if tp+fn else 0)
    F=2*tp/(2*tp+fp+fn) if tp+fp+fn else 0
    return dict(P=round(P,3),R=round(R,3),F1=round(F,3),
                TP=tp,FP=fp,FN=fn,TN=tn)

def win_PRF(mask,truth):
    P,R,F,_=precision_recall_fscore_support(truth,mask,average='binary')
    tn,fp,fn,tp=confusion_matrix(truth,mask).ravel()
    return dict(P=round(P,3),R=round(R,3),F1=round(F,3),
                TP=int(tp),FP=int(fp),FN=int(fn),TN=int(tn))

# ── grid search on validation (flash F1) ───────────────────
best=(0,0,0, -1)
for perc in GRID_IF:
    g_if = score>np.percentile(score[vl],perc)
    for k in GRID_STA_K:
        g_sta = feat[:,0]>sta_mu+k*sta_sd
        for cf in GRID_CF:
            g_cf  = feat[:,3]>cf
            m = (g_sta|g_if)&(g_sta|g_cf)
            F=flash_PRF(m[vl],win_truth[vl])["F1"]
            if F>best[3]:
                best=(perc,k,cf,F)
score_thr=np.percentile(score,best[0])
sta_thr =sta_mu+best[1]*sta_sd
cf_thr  =best[2]
print(f"best grid on val: IF>{best[0]}p  STA>(μ+{best[1]}σ)  CF>{best[2]}")
mask_all=(score>score_thr)|(feat[:,0]>sta_thr)
mask_all=(mask_all)&((feat[:,0]>sta_thr)|(feat[:,3]>cf_thr))

# ── report ─────────────────────────────────────────────────
print("\nWINDOW metrics")
for name,sl in zip(["train","val","test"],[tr,vl,te]):
    print(f" {name:<5}",win_PRF(mask_all[sl],win_truth[sl]))

print("\nFLASH metrics")
for name,sl in zip(["train","val","test"],[tr,vl,te]):
    print(f" {name:<5}",flash_PRF(mask_all[sl],win_truth[sl]))


feat: 100%|██████████████████| 70309/70309 [00:05<00:00, 13075.02it/s]


best grid on val: IF>70p  STA>(μ+2.0σ)  CF>5.5

WINDOW metrics
 train {'P': 0.964, 'R': 0.26, 'F1': 0.41, 'TP': 265, 'FP': 10, 'FN': 753, 'TN': 34124}
 val   {'P': 0.981, 'R': 0.266, 'F1': 0.419, 'TP': 203, 'FP': 4, 'FN': 559, 'TN': 16812}
 test  {'P': 0.973, 'R': 0.26, 'F1': 0.411, 'TP': 144, 'FP': 4, 'FN': 409, 'TN': 17022}

FLASH metrics
 train {'P': 0.975, 'R': 0.907, 'F1': 0.94, 'TP': 39, 'FP': 1, 'FN': 4, 'TN': 35108}
 val   {'P': 1.0, 'R': 0.9, 'F1': 0.947, 'TP': 27, 'FP': 0, 'FN': 3, 'TN': 17548}
 test  {'P': 0.947, 'R': 0.857, 'F1': 0.9, 'TP': 18, 'FP': 1, 'FN': 3, 'TN': 17557}
